# Data Collection and Modelings

## Example API access to NREL.gov
First I demonstrate how to access the data source NREL. After this, I explain how we use this data to calculate plausible PV outage over days, months, and then years for various locations. Finally, I use this data to train a model to predict the PV outage for the future for various locations.

In [25]:
import sys, os
import pandas as pd
import numpy as np
from IPython.display import display

#idk why this fix does stuff
import ssl
import certifi
import certifi
import urllib
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()
context = ssl.create_default_context(cafile=certifi.where())
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())
# ssl._create_default_https_context = ssl._create_unverified_context

In [28]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
lat, lon, year = 33.2164, -118.2437, 2010 #Long is always negative Los Angeles USA: 33.2164, -118.2437
# You must request an NSRDB api key from the link above
api_key = 'i6Pcob8Y47SEHUlKuyjPSu2zozcFv9ne0DuB8aw7'
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,air_temperature,relative_humidity,wind_speed'
# Choose year of data
year = year
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '30'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'
# Your full name, use '+' instead of spaces.
your_name = 'andrew+xavier'
# Your reason for using the NSRDB.
reason_for_use = 'beta+testing'
# Your affiliation
your_affiliation = 'rice+university'
# Your email address
your_email = 'ahx1@rice.edu'
# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'true'

# Declare url string
url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)
# See metadata for specified properties, e.g., timezone and elevation
timezone, elevation = info['Local Time Zone'], info['Elevation']


In [9]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv('https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes), skiprows=2)

# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
print('shape:', df.shape)
df.head()

shape: (17520, 9)


,Year,Month,Day,Hour,Minute,GHI,Temperature,Relative Humidity,Wind Speed
2010-01-01 00:00:00,2010,1,1,0,0,0,15,79.37,2.3
2010-01-01 00:30:00,2010,1,1,0,30,0,15,79.35,2.5
2010-01-01 01:00:00,2010,1,1,1,0,0,15,78.67,2.7
2010-01-01 01:30:00,2010,1,1,1,30,0,15,78.65,3.0
2010-01-01 02:00:00,2010,1,1,2,0,0,15,77.03,3.3


In [7]:
#For LaTeX
print(df.head().to_latex(index=False))  

\begin{tabular}{rrrrrrrrr}
\toprule
 Year &  Month &  Day &  Hour &  Minute &  GHI &  Temperature &  Relative Humidity &  Wind Speed \\
\midrule
 2010 &      1 &    1 &     0 &       0 &    0 &           15 &              79.37 &         2.3 \\
 2010 &      1 &    1 &     0 &      30 &    0 &           15 &              79.35 &         2.5 \\
 2010 &      1 &    1 &     1 &       0 &    0 &           15 &              78.67 &         2.7 \\
 2010 &      1 &    1 &     1 &      30 &    0 &           15 &              78.65 &         3.0 \\
 2010 &      1 &    1 &     2 &       0 &    0 &           15 &              77.03 &         3.3 \\
\bottomrule
\end{tabular}



C:\Users\andre\AppData\Local\Temp\ipykernel_22020\3906604345.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.head().to_latex(index=False))


## Data Processing and Calculating PV Output Using NREL Data
The data is too granular to use as it is. As such, I will generate a new column that calculates the power generation using temperature and GHI levels. Then, I will sum them over each day, then average by month giving average total "power" per day per month. We cannot take the average temperature of each day and the average GHI of each day as seen in the EDA section (the GHI levels and temperature levels do not follow the same patterns.)

### Calculations for Energy:
#### The following is based off the equations for PV output given by Homer: 
https://www.homerenergy.com/products/pro/docs/3.11/how_homer_calculates_the_pv_array_power_output.html

Power output in kW for a solar panel is given by:
$$P = Y_{pv}f_{pv}(\frac{G_T}{G_{T,STC}})[1+\alpha*(T_c - T_{c,STC})]$$

$Y_{pv}$ = the rated capacity of the PV array, meaning its power output under standard test conditions [kW]. Best PV array has value of 400W, but some are as low as 260W (https://www.solaris-shop.com/hanwha-q-cells-q-pro-bfr-g4-260-260w-poly-solar-panel/ https://www.solar.com/learn/how-much-energy-does-a-solar-panel-produce/). Note however, this is by cells! 

$f_{pv}$ = the PV derating factor [%]

$G_T$ = the solar radiation incident on the PV array in the current time step [kW/m2]. 

$G_{T,STC}$ = the incident radiation at standard test conditions [1 kW/m2]

α = the temperature coefficient of power [%/°C]

$T_c$ = the PV cell temperature in the current time step [°C]

$T_{c,STC}$ = the PV cell temperature under standard test conditions [25°C]

#### Simplify

Since we only care about variation in GHI and temperature, we consider $Y_{pv}$, $f_{pv}$ $G_{T,STC}$ to be constants that will only scale our results and as we only care at relative differences in our final output, we can consider these to be set to $1\frac{\%}{m^2}$ for simplicity. As such, our equations simplifies to:

$G_T[1+\alpha*(T_c - T_{c,STC})]$

Note by doing this, we have change to a scalar multiple of the true Power output!

From: https://wiki.openmod-initiative.org/wiki/Standard_test_conditions#:~:text=STC%20is%20an%20industry%2Dwide,5)%20spectrum. 


We know $T_{c,STC}$ is just 25 degrees Celcius
Our equation becomes:
$$G_T[1+\alpha*(T_c - 25\degree C)]$$
$G_T$ is caculated by looking at the radiation incident on the panel which varries by the angle of the sunlight to the panel. However, given that the setup of the solar panel will have either a constant angle or adjust uniformly throughout the day each year and that the sun's path in the sky over each year will follow the same path, we know $G_T$ is proportional to GHI. As such our relative power $P_r$ is calculated by


$$P_r = GHI*[1+\alpha*(T_c - 25\degree C)]$$

Source for $G_T$ from GHI and angles: https://www.homerenergy.com/products/pro/docs/3.11/how_homer_calculates_the_radiation_incident_on_the_pv_array.html

$\alpha$ is given by the manufature of the solar panel and for this example, we will use the value given for Polycrystalline silicon panels: -0.48[%/°C].

Source for $\alpha$ value: https://www.homerenergy.com/products/pro/docs/3.11/pv_temperature_coefficient_of_power.html

The final piece we need is $T_c$. This is the difficult part and will be described below in the Next section

### Getting the temperature of the Cell
From the last section, we know the final piece to our equation to measure the importance of GHI and  ambient temperature is to find the temperature of the cell which is partly a function of the ambient temperature.

$T_c$ is given by: $T_c = T_a + G_T*\frac{\tau\alpha}{U_L}(1-\frac{\eta}{\tau\alpha})$
Where:
	
τ = the solar transmittance of any cover over the PV array [%]

α = the solar absorptance of the PV array [%]

$G_T$ = the solar radiation striking the PV array [kW/m2]

η = the electrical conversion efficiency of the PV array [%]

$U_L$ = the coefficient of heat transfer to the surroundings [kW/m2°C]

$T_c$ = the PV cell temperature [°C]

$T_a$ = the ambient temperature [°C]


$\textbf{Deducing our values}$

$\tau*\alpha$ : 0.9, or 90%

HOMER assumes a value of 0.9 for τα in the equation above, as Duffie and Beckman (1991) suggest. Because the term η/τα is small compared to unity, this assumption does not introduce significant error. Source: Duffie JA, Beckman WA (1991), Solar Engineering of Thermal Processes 2nd edition, Wiley, New York, NY

$\eta$: 0.2, or 20%

From umich, we know a reasonable value for this is 15%-20% in commercial panels. We will use 20% for this project
https://css.umich.edu/publications/factsheets/energy/photovoltaic-energy-factsheet#:~:text=PV%20conversion%20efficiency%20is%20the,that%20is%20converted%20to%20electricity.&text=Though%20most%20commercial%20panels%20have,cells%20with%20efficiencies%20approaching%2050%25.

$G_T$:

GHI

$U_L$: $0.01kW/m^2\degree C$

This is tricky as it depends on the temperature of the ambient temperature air velocity, nu, and solar panel temperature. However, Homer provides a neat solution. We can express $\frac{\tau\alpha}{U_L}$ as $\frac{\tau\alpha}{U_L} = \frac{T_{c,NOCT}- T_{a,NOCT}}{G_{T,NOCT}}$ (https://www.homerenergy.com/products/pro/docs/latest/how_homer_calculates_the_pv_cell_temperature.html).

$T_{c,NOCT}$ = the nominal operating cell temperature [°C] (45-48 $\degree C$) https://www.homerenergy.com/products/pro/docs/latest/pv_nominal_operating_cell_temperature.html. We will set this to 45°C as this is a feature of the cell and will be constant if the same cell type is used. This does restrict us to use one type of cell in comparison.

$T_{a,NOCT}$ = the ambient temperature at which the NOCT is defined [20°C]

$G_{T,NOCT}$ = the solar radiation at which the NOCT is defined [0.8 kW/m2]

Thus, $$\frac{\tau\alpha}{U_L} = \frac{T_{c,NOCT}- T_{a,NOCT}}{G_{T,NOCT}} = (45-20)/0.8 = \frac{32.25}{kW/m2°C}$$ 
and $$T_c = T_a+(GHI*(45-20)/0.8)*(1-20/90)$$

As a result, our final equation is:
$$ 
P_r = GHI*[1-(0.48\%/\degree C)*[T_a + GHI*(\frac{T_{c,NOCT}- T_{a,NOCT}}{G_{T,NOCT}})*(1-\frac{20}{90}) - 25\degree C]]
$$
Note that $P_r$ is measured in kW. We can also scale the whole equation by the Power rating of the cell ($Y_{PV}$). For example, we can consider our best cell rating of 450W. $\textbf{We will scale our results by this (400W) in our code to get us closer to real world results}$ based on https://www.solar.com/learn/how-much-energy-does-a-solar-panel-produce/.


From here, we can finally fill out our relative power metric column for the efficiency of the solar pannels given GHI and ambient air temperature.



In [23]:
def calcPV_example(ghi, temp):
    #ghi: W/m^2
    #temp: Ambient temperature Celcius 
    #returns cell temp and PV output for a 2
    ghi = ghi/1000 #this is because nrel return GHI W/m^2, https://nsrdb.nrel.gov/data-sets/us-data
    t_c = temp+(ghi*(45-20)/0.8)*(1-20/90)
    return t_c, ghi*(1-.0048*(t_c-25))

# We get these from the example locations in LA above
ghi = float(1000.0) #peak outs
temp = float(18.0)
t_c, out= calcPV_example(ghi,temp)
print("Cell temp given ambient is 18 degrees:",t_c)
print("Efficency loss percentage given ambient is 18 degrees",(t_c-25)*0.48,"%")
print("Efficiency of max output:",out, ", kWh for 4.5 hours given ambient is 18 degrees:",out*.400*4.5,"kWh/day")

print("\n")

ghi = float(1000.0) #peak outs
temp = float(0.0)
t_c, out= calcPV_example(ghi,temp)
print("Cell temp given ambient is 18 degrees:",t_c)
print("Efficency loss percentage given ambient is 18 degrees",(t_c-25)*0.48,"%")
print("Efficiency of max output:",out, ", kWh for 4.5 hours given ambient is 18 degrees:",out*.400*4.5,"kWh/day")


Cell temp given ambient is 18 degrees: 42.30555555555556
Efficency loss percentage given ambient is 18 degrees 8.306666666666667 %
Efficiency of max output: 0.9169333333333334 , kWh for 4.5 hours given ambient is 18 degrees: 1.6504800000000004 kWh/day


Cell temp given ambient is 18 degrees: 24.305555555555557
Efficency loss percentage given ambient is 18 degrees -0.33333333333333254 %
Efficiency of max output: 1.0033333333333334 , kWh for 4.5 hours given ambient is 18 degrees: 1.8060000000000003 kWh/day


From our example using temp of 18 degrees in ambient temperature and peak sunlight hours with no cloud cover (~1000W from online sources previously listed), we get 1.6504kWh/day if our unit operates only during peak hours for 4.5 hours. These parameters are used just to make sure we match the example data given often by online resources for solar panel installation in homes. Using https://www.solar.com/learn/how-much-energy-does-a-solar-panel-produce/ as our example, we can see that we are within reasonable values. solar.com gives 1.800kWh/day but assumes maximum efficiency. Our model accounts for cell temperature calculations and thus can tell we have a 8.3% efficiency decrease thus giving a lower PV output. If we set our ambient temperature $T_A$ to 0 degrees celcius, we can see cell temperature drops to ~25 degrees, the temperature the 400W output was measured by the manufacture and thus we perfectly match solar.coms results at 1.8kWh/day! We have validated our results and calculations.

From here, we are good to go! Lets apply this calculation by taking the GHI and temperature and multiplying it by 1 for each row to get the kWh/hr (in practice we just leave it). We then can sum the values for each day to get kWh/day. For our training data, this is sufficient. For our targets (y values in our regression) we calculate: Average kWh/day for the year, kWh/year, and kWh/month (sum every 30 days, ignore last 5 days).

## Preprocess data for training:
We can generate 15 data points for each long/lat pair (training is last 5 years, prediction is next year). To get a reasonable dataset, we will do this for 100 different long/lat pairs within a region. This gives us 1500 datapoints. We need to make 2000 API calls thus we will need to automate this step. First, lets process our data for one year using the example data for Los Angeles as given.

In [6]:
#Example for one year using API
from APIs.data_processing import getData, addPV, hoursToDays, daysToMonths, hoursToMonths
# Parameters
api_key = 'KGQPxhHwezmhBj96AirFO6eKuzfP5DvQI7gFDWOk'
your_name = 'andrew+xavier'
your_affiliation = 'columbia+university'
your_email = 'ahx2001@columbia.edu'
lat = 33.2164
lon = -118.2437
year = 2020
cell_rating = 0.400

#Data processing
df = getData(api_key, your_name, your_affiliation, your_email, lat=lat, lon = lon, year=year) #get raw data from nrel
df_pv = addPV(df,cell_rating=cell_rating) #Add PV calculated column
df_pv_day = hoursToDays(df_pv) # Convert to day level granularity
df_pv_month = daysToMonths(df_pv_day)
print("Daily for Los Angeles \n",df_pv_day.head()) #display results
print("Monthly for Los Angeles\n", hoursToMonths(df_pv).head())#faster version check


Daily for Los Angeles 
    Year  Month  Day  Temp mean    GHI mean  PV kWh/day
0  2020      1    1  16.120833  142.291667    1.348385
1  2020      1    2  16.075000  134.666667    1.278246
2  2020      1    3  17.387500  139.291667    1.313410
3  2020      1    4  16.416667  144.208333    1.364167
4  2020      1    5  16.741667  150.125000    1.415023
Monthly for Los Angeles
    Year  Month  Temp mean    GHI mean  PV kWh/month
0  2020      1  16.049462  134.263441     39.480669
1  2020      2  15.986310  182.773810     47.972402
2  2020      3  15.697177  203.498656     58.843999
3  2020      4  16.727361  251.477778     69.271297
4  2020      5  18.329032  298.303763     83.761602


In [7]:
#Generate training data test:
from APIs.data_processing import getData, addPV, hoursToDays, daysToMonths, dataByLocation

# Parameters
api_key = 'KGQPxhHwezmhBj96AirFO6eKuzfP5DvQI7gFDWOk'
your_name = 'andrew+xavier'
your_affiliation = 'columbia+university'
your_email = 'ahx2001@columbia.edu'
lat = 33.2164
lon = -118.2437
cell_rating = 0.400
end_year = 2001 #data is 1998-end_year

import pandas as pd
data = dataByLocation(api_key, your_name, your_affiliation, your_email, lat, lon, cell_rating,end_year)
file_name = "API_data/lon_lat{location}.csv".format(location = lon+lat)
data.to_csv(file_name, index = False)

Now we have made sure all our API functions are working and online! Time to get data for many locations. These locations are the ones described in "1 Map Visualization and Data Selection.ipynb". The data points are saved in API_data.

In [4]:
#Generate training data across all points:
""" 
Runtime is ~1min 30sec/point. Over 100 points takes around 150 minutes
Adjust the overal

"""
#Change these two parameters as needed and the start and stop points in points below:
file_name = "API_data/x_train_156.csv" #Change number to end point index
lastpoint = 142 #set same as start time

import pandas as pd
from APIs.data_processing import getData, addPV, hoursToDays, daysToMonths, dataByLocation
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# Parameters
api_key = 'i6Pcob8Y47SEHUlKuyjPSu2zozcFv9ne0DuB8aw7'
your_name = 'andrew+xavier'
your_affiliation = 'columbia+university'
your_email = 'ahx1@rice.edu'
cell_rating = 0.400
end_year = 2021 #data is 1998-end_year, max is 2021


#generate lon lat points:
lon = -104.0000 #for all values
points = []
#lat starts at 20.0000 and incriments by .045 each step
for lat in range(20000, 27000, 45):
    points.append([format(lat/1000,'.4f'), format(lon,'.4f')])
# print(points[1], "exected 20.0450, -104.0000")


#Generate all our points! This will run for an hour or so
x_train = pd.DataFrame(columns=["Year", "Month", "Temp mean", "GHI mean", "PV kWh/month"])
for point in points[142:len(points)]: #Update this before running
    lat = point[0]
    lon = point[1]
    data = dataByLocation(api_key, your_name, your_affiliation, your_email, lat, lon, cell_rating, end_year=end_year)
    x_train = pd.concat([x_train, data])
    x_train.to_csv(file_name, index = False)
    lastpoint += 1
    print(lastpoint)
print("num training points", len(x_train))


143
144
145
146
147
148
149
150
151
152
153
154
155
156
num training points 3864


In [3]:
#Concat data for final set
import pandas as pd
df = pd.read_csv("API_data/x_train_10.csv")
df = pd.concat([df, pd.read_csv("API_data/x_train_50.csv")])
df = pd.concat([df, pd.read_csv("API_data/x_train_59.csv")])
df = pd.concat([df, pd.read_csv("API_data/x_train_120.csv")])
df = pd.concat([df, pd.read_csv("API_data/x_train_142.csv")])
df = pd.concat([df, pd.read_csv("API_data/x_train_156.csv")])
df.to_csv("API_data/x_train.csv",index=False)

## Modeling
I use a simple linear regression model to create my predictions with my features being previous values and my targets being the relative PV out values for months 1,2, and 3 years in the future. Note these would be 3 different models as this generates 3 different types of training data pairs!

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

########################################################################################################################
#PARAMETERS:
years_ahead = 3 # How far in the future we should predict in years (1-22). Ex 1 means predict 1 values for 1 year in the future.
num_years = 10 #Number of years used to predict future values. Ranges from 1 to 23 minus the years_ahead 

##Combinations to try: 
# Use 7 years to predict, predict 2 years ahead.
# Use 10 years to predict, predict 4 years ahead

########################################################################################################################


x_set = [] #our inputs to train
y_set = [] #our inputs to train

df = pd.read_csv("API_data/x_train.csv") #read in our data
########
#TO DO:
#Create train-test split to validate on later. Right now just training
########


#generate lon lat points NEED TO UPDATE THIS FOR TRAIN TEST SPLIT!:
lon = -104.0000 #for all values
points = []
#lat starts at 20.0000 and incriments by .045 each step
for lat in range(20000, 27000, 45):
    points.append([format(lat/1000,'.4f'), format(lon,'.4f')])

for location in points: #iterate over each point to build point prediction pairs
    df_loc = df.loc[(df['lat'] == float(location[0])) & (df['lon'] == float(location[1]))] #select location
    X = df_loc.values
    y = df_loc["PV kWh/month"].tolist()
    for i in range(len(X)//12 - num_years-years_ahead+1):
        x_set.append(X[i*12:i*12+12*num_years].flatten())
        y_set.append(y[i*12+12*num_years+12*(years_ahead-1):i*12+12*num_years+12+12*(years_ahead-1)])

#train test split:
x_train, x_test, y_train, y_test = train_test_split(x_set, y_set, test_size=0.20,shuffle=True)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)


print("shape of x_train",x_train.shape)
print("shape of y_train", y_train.shape)
# print("num of points to predict used correct?", x_train.shape[1]/(12*7) == num_years)
print("\n")
#train model
reg = LinearRegression().fit(x_train,y_train)
print("score on training (R2), closer to 1 is better:", reg.score(x_train,y_train))
print("score on testing (R2), closer to 1 is better:", reg.score(x_test,y_test))

#Testing
x_test = df.values[-num_years*12:].flatten()
print("Predictions for power output PV (kWh/month) for each month in", years_ahead,"year(s) after", int(x_test[-7]),"in location lat long =", x_test[-2:]) 
prediction = reg.predict([x_test])[0]
months = ["Jan", "Feb", "March", "April", "May", "June", "July", "Aug", "Sept", "Oct", "Nov", "Dec"]
for index, month in enumerate(months):
    print(month,prediction[index],"kWh/Month")


shape of x_train (1372, 840)
shape of y_train (1372, 12)


score on training (R2), closer to 1 is better: 0.981762714971763
score on testing (R2), closer to 1 is better: 0.9663493437929852
Predictions for power output PV (kWh/month) for each month in 3 year(s) after 2020 in location lat long = [  26.975 -104.   ]
Jan 55.1524658203125 kWh/Month
Feb 62.505035400390625 kWh/Month
March 82.550537109375 kWh/Month
April 77.4635009765625 kWh/Month
May 98.2757568359375 kWh/Month
June 75.85595703125 kWh/Month
July 88.673828125 kWh/Month
Aug 117.50048828125 kWh/Month
Sept 52.69677734375 kWh/Month
Oct 49.662109375 kWh/Month
Nov 52.65119934082031 kWh/Month
Dec 35.8912353515625 kWh/Month
